In [3]:
import requests
import json
from lxml import html
from ncbiumls.authorization import authorization
from time import sleep


class UMLS(authorization):
    def __init__(self, api_key='e9990d5d-ff93-40f1-9f60-29ce7cf2950c'):
        super().__init__(str(api_key))

    def askTerm(self, term, inputType='atom', includeObsolete='false', includeSuppressible='false',
                returnIdType='concept', searchType='words', pageNumber=1, pageSize=25):

#   TODO: Add option like dump, if enabled - save data, current page, page size
#   TODO: Write some messages on succesfull access, like pageNumber etc.

        """
        Description: retrives all records associted with the term, returns list of dicts,
                     each dict have keywords name,uri,ui,rootSource.


        :param term:                A human readable term, such as 'gestatational diabetes', or a code from a source
                                    vocabulary, such as 11687002 from SNOMEDCT_US.

        :param inputType:           Specifies the data type you are using as your search parameter.
                                    Valid values:  'atom', 'code','sourceConcept','sourceDescriptor','sourceUi','tty'
                                    Default value: 'atom'
                                    Usage Note:     Use 'sourceUi' if you aren't sure if the identifier you're providing
                                                    is a code, source concept, or source descriptor. Using 'tty' is for
                                                    advanced use cases and will extract codes from a specified
                                                    vocabulary according to term type.

        :param includeObsolete:     Return content that is a result of matches on obsolete terms.
                                    Valid values:   true or false
                                    Default value: 'false'

        :param includeSuppressible: Return content that is a result of matches on suppressible terms.
                                    Valid values:   true or false
                                    Default value: 'false'

        :param returnIdType:        Specifies the type of identifier you wish to retrieve.
                                    Valid values:   true or false
                                    Default value: 'false'
                                    Usage Note:     Use 'code','sourceConcept', 'sourceDescriptor', or 'sourceUi' if you
                                                    prefer source-asserted identifiers rather than CUIs in your search
                                                    results.

        :param searchType:          Type of search you wish to use.
                                    Valid values:  'exact','words','leftTruncation', 'rightTruncation','approximate',
                                                   'normalizedString'
                                    Default value: 'words'
                                    Usage Note:     Use 'exact' when using inputType = 'code', 'sourceConcept',
                                                   'sourceDescriptor', or 'sourceUi'.

        :param pageNumber:          Whole number that specifies which page of results to fetch
                                    Valid values:   1,2,3, etc
                                    Default value:  1

        :param pageSize:            Whole number that specifies the number of results to include per page.
                                    Valid values:   1,2,3, etc
                                    Default value:  25

        :return:
        """
        # main url and parameters
        url = 'https://uts-ws.nlm.nih.gov/rest/search/current'
        params = {'ticket': '',
                  'string': str(term),
                  'inputType': inputType,
                  'includeObsolete': includeObsolete,
                  'includeSuppressible': includeSuppressible,
                  'returnIdType': returnIdType,
                  'searchType': searchType,
                  'pageNumber': pageNumber,
                  'pageSize': pageSize}

        #varible to store received data
        data = []

        #in case specific page is not provided - grab all results starting from page 1
        if pageNumber == 1:

            #function is guaranteed to return results so ask cycle will roll infinitely
            while True:

                #get a service ticket
                params['ticket'] = self.getST()

                #access page
                get = requests.get(url=url, params=params)
                try:

                    #if all the results obtained return gathered data
                    if get.json()['result'] == {'results': [{'ui': 'NONE', 'name': 'NO RESULTS'}],
                                                  'classType': 'searchResults'}:
                        return data

                    #else grab some more data
                    else:
                        for item in get.json()['result']['results']:
                            data.append(item)
                        params['pageNumber'] += 1

                #if server returned error - try to print that error, wait and retry request
                except Exception as exp:
                    response = html.fromstring(data.text)
                    print(response.xpath('.//p/b[contains(text(),"description")]/following::u')[0].text,
                          ', current page number = ' + str(pageNumber),
                          ', current page size = ' + str(pageSize))

                #to not overlode server pause a bit
                sleep(0.5)

        #else access specific page
        else:
            params['ticket'] = self.getST()
            get = requests.get(url=url, params=params)
            try:
                return get.json()['result']['results']
            except Exception as exp:
                response = html.fromstring(data.text)
                print(response.xpath('.//p/b[contains(text(),"description")]/following::u')[0].text,
                      ', current page number = ' + str(pageNumber),
                      ', current page size = ' + str(pageSize))



class CUI(authorization):
    def __init__(self, api_key='e9990d5d-ff93-40f1-9f60-29ce7cf2950c'):
        super().__init__(api_key=api_key)

    def CUIGeneral(self, cui):
        """
        :param cui: The Concept Unique Identifier for a Metathesaurus concept
        :return: Returns dictionary processed from JSON, example:
                {
                    "pageSize": 25,
                    "pageNumber": 1,
                    "pageCount": 1,
                    "result": {
                        "classType": "Concept",
                        "ui": "C0009044",
                        "suppressible": false,
                        "dateAdded": "09-30-1990",
                        "majorRevisionDate": "08-18-2015",
                        "status": "R",
                        "semanticTypes": [
                            {
                                "name": "Injury or Poisoning",
                                "uri": "https://uts-ws.nlm.nih.gov/rest/semantic-network/2015AB/TUI/T037"
                            }
                        ],
                        "atomCount": 63,
                        "attributeCount": 0,
                        "cvMemberCount": 0,
                        "atoms": "https://uts-ws.nlm.nih.gov/rest/content/2015AB/CUI/C0009044/atoms",
                        "definitions": "NONE",
                        "relations": "https://uts-ws.nlm.nih.gov/rest/content/2015AB/CUI/C0009044/relations",
                        "defaultPreferredAtom": "https://uts-ws.nlm.nih.gov/rest/content/
                                                        2015AB/CUI/C0009044/atoms/preferred",
                        "relationCount": 5,
                        "name": "Closed fracture carpal bone"
                    }
                }
        """
        stTicket = self.getST()
        url = 'https://uts-ws.nlm.nih.gov/rest/content/current/CUI/' + str(cui)
        params = {'ticket': stTicket}
        data = requests.get(url=url, params=params)
        try:
            return data.json()

        except Exception as exp:
            response = html.fromstring(data.text)
            return response.xpath('.//p/b[text()="message"]/u')

    def CUISemanticTypes(self, cui):
        """
        :param cui: The Concept Unique Identifier for a Metathesaurus concept
        :return: Returns dictionary processed from JSON, example:
                {
                    "pageSize": 25,
                    "pageNumber": 1,
                    "pageCount": 1,
                    "result": {
                        "classType": "Concept",
                        "ui": "C0009044",
                        "suppressible": false,
                        "dateAdded": "09-30-1990",
                        "majorRevisionDate": "08-18-2015",
                        "status": "R",
                        "semanticTypes": [
                            {
                                "name": "Injury or Poisoning",
                                "uri": "https://uts-ws.nlm.nih.gov/rest/semantic-network/2015AB/TUI/T037"
                            }
                        ],
                        "atomCount": 63,
                        "attributeCount": 0,
                        "cvMemberCount": 0,
                        "atoms": "https://uts-ws.nlm.nih.gov/rest/content/2015AB/CUI/C0009044/atoms",
                        "definitions": "NONE",
                        "relations": "https://uts-ws.nlm.nih.gov/rest/content/2015AB/CUI/C0009044/relations",
                        "defaultPreferredAtom": "https://uts-ws.nlm.nih.gov/rest/content/
                                                        2015AB/CUI/C0009044/atoms/preferred",
                        "relationCount": 5,
                        "name": "Closed fracture carpal bone"
                    }
                }
        """
        stTicket = self.getST()
        url = 'https://uts-ws.nlm.nih.gov/rest/content/current/CUI/' + str(cui)
        params = {'ticket': stTicket}
        data = requests.get(url=url, params=params)
        try:
            return data.json()['result']['semanticTypes']

        except Exception as exp:
            response = html.fromstring(data.text)
            return response.xpath('.//p/b[text()="message"]/u')

    def CUIAtoms(self, cui, pageNumber=1, pageSize=25):
        stTicket = self.getST()
        url = 'https://uts-ws.nlm.nih.gov/rest/content/current/CUI/' + str(cui) + '/atoms'
        params = {'ticket': stTicket,
                  'pageNumber': pageNumber,
                  'pageSize': pageSize}
        data = requests.get(url=url, params=params)
        pass

    def CUIDefenitions(self, cui, pageNumber=1, pageSize=25):
        stTicket = self.getST()
        url = 'https://uts-ws.nlm.nih.gov/rest/content/current/CUI/' + str(cui) + '/definitions'
        params = {'ticket': stTicket,
                  'pageNumber': pageNumber,
                  'pageSize': pageSize}
        data = requests.get(url=url, params=params)
        pass

    def CUIRelations(self, cui, pageNumber=1, pageSize=25):
        stTicket = self.getST()
        url = 'https://uts-ws.nlm.nih.gov/rest/content/current/CUI/' + str(cui) + '/relations'
        params = {'ticket': stTicket,
                  'pageNumber': pageNumber,
                  'pageSize': pageSize}
        data = requests.get(url=url, params=params)
        pass

In [13]:
data_umls = UMLS()

print(data_umls.getST())
print(data_umls.tgt.split('/')[-1])

ST-2449248-9MBklLIB7AdLMulMIpEy-cas
TGT-1008454-keND7cN9krcD45QJg4kuACxvpGosWDk4QcMFqcAXXTctomz31R-cas


In [27]:
with open('data/1990.json') as data_file:
    data = json.load(data_file)

sorted(data['chemicals'].items(), key=lambda x: x[1], reverse=True)

[('a', 1626),
 ('l', 1084),
 ('t', 542),
 ('i', 542),
 ('e', 542),
 (' ', 542),
 (']', 542),
 ('v', 542),
 ('[', 542),
 ('b', 542),
 ('o', 542),
 ('n', 542),
 ('amyloid', 102),
 ('amyloid beta-peptides', 71),
 ('protein precursors', 70),
 ('amyloid beta-protein precursor', 63),
 ('nerve tissue proteins', 47),
 ('tau proteins', 40),
 ('microtubule-associated proteins', 39),
 ('antibodies, monoclonal', 32),
 ('rna, messenger', 31),
 ('choline o-acetyltransferase', 26),
 ('neurotransmitter agents', 23),
 ('protease inhibitors', 20),
 ('nerve growth factors', 20),
 ('tacrine', 20),
 ('acetylcholinesterase', 19),
 ('aluminum', 18),
 ('dna', 16),
 ('receptors, neurotransmitter', 15),
 ('glutamates', 14),
 ('aminoacridines', 14),
 ('biomarkers', 13),
 ('epitopes', 13),
 ('glial fibrillary acidic protein', 12),
 ('antibodies', 12),
 ('neuropeptides', 12),
 ('physostigmine', 12),
 ('hydrocortisone', 11),
 ('acetylcholine', 11),
 ('ubiquitins', 11),
 ('receptors, cholinergic', 10),
 ('amino acid

In [24]:
with open('data/1993.json') as data_file:
    data = json.load(data_file)

sorted(data['chemicals'].items(), key=lambda x: x[1], reverse=True)

[('a', 1500),
 ('l', 1000),
 ('i', 500),
 (']', 500),
 ('o', 500),
 ('[', 500),
 ('b', 500),
 (' ', 500),
 ('e', 500),
 ('t', 500),
 ('v', 500),
 ('n', 500),
 ('amyloid beta-protein precursor', 133),
 ('amyloid beta-peptides', 126),
 ('tau proteins', 61),
 ('tacrine', 39),
 ('amyloid', 34),
 ('antibodies, monoclonal', 33),
 ('apolipoproteins e', 31),
 ('genetic markers', 31),
 ('nerve tissue proteins', 30),
 ('aluminum', 30),
 ('peptide fragments', 27),
 ('cholinesterase inhibitors', 25),
 ('acetylcholine', 24),
 ('rna, messenger', 21),
 ('nerve growth factors', 21),
 ('biomarkers', 20),
 ('dna', 19),
 ('receptors, muscarinic', 18),
 ('choline o-acetyltransferase', 18),
 ('epitopes', 17),
 ('apolipoprotein e4', 16),
 ('glucose', 16),
 ('calcium', 15),
 ('neurotransmitter agents', 14),
 ('parasympathomimetics', 12),
 ('protein kinases', 11),
 ('physostigmine', 11),
 ('peptides', 11),
 ('codon', 10),
 ('gtp-binding proteins', 10),
 ('ubiquitins', 10),
 ('glial fibrillary acidic protein',

In [ ]:
import os



In [25]:
with open('data/1994.json') as data_file:
    data = json.load(data_file)
    meshes = data['chemicals']

sorted(meshes.items(), key=lambda x: x[1], reverse=True)

[('a', 2028),
 ('l', 1352),
 ('i', 676),
 ('e', 676),
 ('o', 676),
 ('[', 676),
 ('b', 676),
 (' ', 676),
 ('t', 676),
 (']', 676),
 ('v', 676),
 ('n', 676),
 ('amyloid beta-peptides', 165),
 ('amyloid beta-protein precursor', 148),
 ('apolipoproteins e', 114),
 ('tau proteins', 85),
 ('nerve tissue proteins', 68),
 ('amyloid', 54),
 ('peptide fragments', 49),
 ('tacrine', 49),
 ('antibodies, monoclonal', 46),
 ('apolipoprotein e4', 35),
 ('rna, messenger', 32),
 ('calcium', 31),
 ('biomarkers', 29),
 ('aluminum', 25),
 ('organotechnetium compounds', 23),
 ('oximes', 21),
 ('nerve growth factors', 21),
 ('technetium tc 99m exametazime', 21),
 ('glucose', 20),
 ('neurofilament proteins', 19),
 ('choline o-acetyltransferase', 19),
 ('dna', 18),
 ('free radicals', 18),
 ('recombinant proteins', 18),
 ('acetylcholine', 17),
 ('epitopes', 15),
 ('endopeptidases', 15),
 ('receptors, muscarinic', 14),
 ('glial fibrillary acidic protein', 14),
 ('acetylcholinesterase', 13),
 ('neurotransmitter

In [26]:
with open('data/2007.json') as data_file:
    data = json.load(data_file)
    meshes = data['chemicals']

sorted(meshes.items(), key=lambda x: x[1], reverse=True)

[('a', 3102),
 ('l', 2068),
 (' ', 1034),
 ('b', 1034),
 ('t', 1034),
 ('n', 1034),
 (']', 1034),
 ('o', 1034),
 ('[', 1034),
 ('e', 1034),
 ('v', 1034),
 ('i', 1034),
 ('amyloid beta-peptides', 636),
 ('amyloid beta-protein precursor', 245),
 ('peptide fragments', 211),
 ('cholinesterase inhibitors', 200),
 ('tau proteins', 198),
 ('biomarkers', 152),
 ('amyloid precursor protein secretases', 129),
 ('amyloid beta-protein (1-42)', 124),
 ('apolipoproteins e', 117),
 ('apolipoprotein e4', 117),
 ('presenilin-1', 91),
 ('piperidines', 79),
 ('nootropic agents', 78),
 ('indans', 73),
 ('neuroprotective agents', 72),
 ('donepezil', 69),
 ('amyloid', 67),
 ('antipsychotic agents', 61),
 ('nerve tissue proteins', 58),
 ('amyloid beta-protein (1-40)', 55),
 ('cholesterol', 52),
 ('radiopharmaceuticals', 51),
 ('antioxidants', 50),
 ('phenylcarbamates', 47),
 ('aspartic acid endopeptidases', 47),
 ('rivastigmine', 45),
 ('rna, messenger', 44),
 ('enzyme inhibitors', 43),
 ('genetic markers', 

In [14]:
# m_term = 'proteins'
# m_term = 'alcoholism genetics'
# m_term = 'alzheimer disease'
# m_term = 'Alcoholism/genetics'
# m_term = 'kcnq1'
# m_term = 'amyloid beta peptides'
# m_term = 'potassium channels'
# m_term = 'Alzheimer'
# m_term = 'Pharmacologic Substance'
m_term = 'n(6)-cyclohexyladenosine'

m_dict_uis = {m_term: []} # term : ui
m_dict_ui_by_name = {}    # ui : term
m_dict_term_by_ui = {}

# res = data.askTerm(m_term, pageSize=100000, searchType='approximate')
res = data_umls.askTerm(m_term, pageSize=100000, searchType='exact')
# res = data.askTerm(m_term, pageSize=100000, inputType='tty', searchType='exact')

for i in res:
    print(i)
    ui = i['ui']
    name = i['name']
    if ui not in m_dict_ui_by_name:
        m_dict_ui_by_name[ui] = []
    m_dict_ui_by_name[ui].append(name)

    m_dict_uis[m_term].append(ui)
    if ui not in m_dict_term_by_ui:
        m_dict_term_by_ui[ui] = []
    m_dict_term_by_ui[ui].append(m_term)

print('--> m_dict_uis')
print(m_dict_uis)
print()

print('--> m_dict_term_by_ui')
print(m_dict_term_by_ui)
print()

print('--> m_dict_ui_by_name')
print(m_dict_ui_by_name)
print()

m_cui = CUI()

m_dict_sem_types = {} # ui : [name1, name2, ...]
m_dict_uis_by_st = {} # sem_type : [ui1, ui2, ...]

for ui in m_dict_uis[m_term]:
    query_res = m_cui.CUIGeneral(ui)
    print('----> ui: {}'.format(ui))
    for sem_type in query_res['result']['semanticTypes']:
        st_name = sem_type['name']

        if st_name not in m_dict_uis_by_st:
            m_dict_uis_by_st[st_name] = []
        m_dict_uis_by_st[st_name].append(ui)

        if ui not in m_dict_sem_types:
            m_dict_sem_types[ui] = []
        m_dict_sem_types[ui].append(st_name)
        print(sem_type)

print()
print('-----> m_dict_sem_types')
print(m_dict_sem_types)

print()
print('-----> m_dict_uis_by_st')
print(m_dict_uis_by_st)



{'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2016AB/CUI/C0067223', 'name': 'N(6)-cyclohexyladenosine', 'ui': 'C0067223', 'rootSource': 'MSH'}
--> m_dict_uis
{'n(6)-cyclohexyladenosine': ['C0067223']}

--> m_dict_term_by_ui
{'C0067223': ['n(6)-cyclohexyladenosine']}

--> m_dict_ui_by_name
{'C0067223': ['N(6)-cyclohexyladenosine']}

----> ui: C0067223
{'name': 'Nucleic Acid, Nucleoside, or Nucleotide', 'uri': 'https://uts-ws.nlm.nih.gov/rest/semantic-network/2016AB/TUI/T114'}

-----> m_dict_sem_types
{'C0067223': ['Nucleic Acid, Nucleoside, or Nucleotide']}

-----> m_dict_uis_by_st
{'Nucleic Acid, Nucleoside, or Nucleotide': ['C0067223']}
